In [70]:
import math
import random
import json
from sklearn import linear_model
import sklearn
import csv
import gzip
import string
import copy
import scipy
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [71]:
#Helpful functions
def readGz(path):
    for l in gzip.open(path, 'rb'):
        yield eval(l)

In [72]:
#Exploring the data
reviewSmall = [] #Users and a list of games they have reviewed
userToItem = [] #Users and a list of games they have downloaded
#User id's ^ directly correlated 

reviewBig = [] #Random assortment of reviews 
itemMetaData = [] #Data about each individual game
bundleData = [] #Data about each bundle
for d in readGz('../australian_user_reviews.json.gz'):
    reviewSmall.append(d)
    
for d in readGz('../australian_users_items.json.gz'):
    userToItem.append(d)
    
for d in readGz('../bundle_data.json.gz'):
    bundleData.append(d)
    
for d in readGz('../steam_games.json.gz'):
    itemMetaData.append(d)

# i = 0
# for d in readGz('../steam_reviews.json.gz'):
#     if i < 50000:
#         reviewBig.append(d)
#     else:
#         break
#     i+=1

In [73]:
relevantUserToItem = []
for u in userToItem:
    if len(u['items'])>=2:
        relevantUserToItem.append(u)

In [74]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [75]:
#Creating train and test sets
train=[]
test=[]
lengths=[]
item_lens=[]
for user in relevantUserToItem:
    items = user['items']
    trainIndex = round(0.7*len(items))
    item_lens.append(len(items))
    lengths.append(len(items)-round(0.7*len(items)))
    items_train = items[:trainIndex]
    items_test = items[trainIndex:]
    objTrain = {'user_id':user['user_id'], 'items': items_train}
    objTest = {'user_id':user['user_id'], 'items': items_test}
    train.append(objTrain)
    test.append(objTest)
    

In [76]:
ItemsForUser=defaultdict(set)
for user in train:
    for item in user['items']:
        ItemsForUser[user['user_id']].add(item['item_id'])

In [77]:
UsersForItem = defaultdict(set)
for user in train:
    for item in user['items']:
        UsersForItem[item['item_id']].add(user['user_id'])

In [78]:
Top10ItemsForUser=defaultdict(list)
for d in train:
    user_id = d['user_id']
    items = d['items']
    items.sort(key=lambda x: x['playtime_forever'], reverse = True)
    itemids = [d['item_id'] for d in items]
    Top10ItemsForUser[user_id] = itemids[:10]

In [79]:
def mostSimilar(u, N):
    similarities = []
    items = set(Top10ItemsForUser[u])
    for u2 in ItemsForUser:
        if u2 == u: continue
        sim = Jaccard(items, set(Top10ItemsForUser[u2]))
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,u2))
    similarities.sort(reverse=True)
#     return similarities[:N]
    return [u[1] for u in similarities[:N]]

In [27]:
list(Top10ItemsForUser.keys())[1]

'js41637'

In [28]:
mostSimilar('js41637', 5)

['visforveni',
 'chocolatecravinghobo',
 'carlospicy_weiner',
 'Thatishayden',
 'R87']

In [103]:
top5SimilarUsers = defaultdict(list)
for d in train[:1000]:
    user_id = d['user_id']
    top5SimilarUsers[user_id] = mostSimilar(user_id, 5)

In [104]:
df = pd.DataFrame(top5SimilarUsers)
df.to_csv('MostSimilarUsersfirst1000.csv')

In [105]:
ItemsForUserTest=defaultdict(set)
for user in test:
    for item in user['items']:
        ItemsForUserTest[user['user_id']].add(item['item_id'])

In [106]:
top5SimilarUsersTest = defaultdict(list)
recommendations=[]
reclengths = []
for d in test[:1000]:
    userid = d['user_id']
    top5similar = top5SimilarUsers[userid]
    top50recs = Top10ItemsForUser[top5similar[0]]+Top10ItemsForUser[top5similar[1]]+Top10ItemsForUser[top5similar[2]]+Top10ItemsForUser[top5similar[3]]+Top10ItemsForUser[top5similar[4]]
    potentialrecs = set(top50recs)-ItemsForUser[userid]
    reclengths.append(len(potentialrecs))
    if len(potentialrecs)>20:
        potentialrecs = set(random.sample(potentialrecs, 20))
    recommendations.append(potentialrecs)

In [107]:
precisions=[]
for i, d in enumerate(test[:30]):
    currec = recommendations[i]
    userid = d['user_id']
    userTestItems = ItemsForUserTest[userid]
    precisions.append(len(userTestItems.intersection(currec))/len(currec))

In [108]:
np.mean(np.array(precisions)>0)

0.4666666666666667